In [45]:
import sys; sys.path.append('common/');
from helper import *

DATA_PATH = '/projects/persuasionforgood-master/Face_acts/dialogue_act_prediction/resisting-persuasion/data'

df = pd.read_csv(DATA_PATH+'/Final_annotations.csv')
df = df.rename(columns={"Our Label": "fine_labels"})

for index, row in df.iterrows():
    if row['B4']==1 and row['fine_labels']!= row['fine_labels']:
        print(index, row)


fine_labels = [i for i in list(set(df['fine_labels'])) if i==i]

fine2coarse_dict = {}
fine2coarse_dict['deflect-responsibility'] = 'contesting'
fine2coarse_dict['organization-inquiry']   = 'contesting'#'biased-processing'
fine2coarse_dict['attack-credibility']     = 'contesting'
fine2coarse_dict['self-pity']              = 'empowerment'
fine2coarse_dict['nitpicking']             = 'contesting'
fine2coarse_dict['direct-rejection']       = 'avoidance'
fine2coarse_dict['personal-choice']        = 'empowerment'
fine2coarse_dict['delay-tactic']           = 'avoidance'
fine2coarse_dict['hesitance']              = 'avoidance'
fine2coarse_dict['not-a-strategy']         = 'not-a-strategy'

fine2resistance_dict = {}
fine2resistance_dict['deflect-responsibility'] = 'counter-argumentation'
fine2resistance_dict['organization-inquiry']   = 'source-degradation' # biased-processing
fine2resistance_dict['attack-credibility']     = 'source-degradation'
fine2resistance_dict['self-pity']              = 'empowerment'
fine2resistance_dict['nitpicking']             = 'counter-argumentation'
fine2resistance_dict['direct-rejection']       = 'self-assertion'
fine2resistance_dict['personal-choice']        = 'attitude-bolstering'
fine2resistance_dict['delay-tactic']           = 'avoidance'
fine2resistance_dict['hesitance']              = 'avoidance'
fine2resistance_dict['not-a-strategy']         = 'not-a-strategy'

coarse_labels     = []
resistance_labels = []

for i, row in df.iterrows():
    label = row['fine_labels']
    if label in fine2coarse_dict:
        coarse_labels.append(fine2coarse_dict[label])
        resistance_labels.append(fine2resistance_dict[label])
    else:
        resistance_labels.append('None')
        coarse_labels.append('None')
        
df['coarse_labels'] = coarse_labels
df['resistance_labels'] = resistance_labels

df.to_csv(DATA_PATH+'/data.csv')
info_df = pd.read_csv(DATA_PATH+'/300_info.csv',sep=',')

sincere_donors_ids=set()
sincere_nondonors_ids=set()
for index, row in info_df.iterrows():
    did= row['B2']
    role=str(row['B4'])
    prop_amt=float(row['B5'])
    amt= float(row['B6'])
    if role =='1':
        if amt>0 and prop_amt<= amt:
            sincere_donors_ids.add(did)
        elif amt==0 and (prop_amt== 0 or math.isnan(prop_amt)):
            sincere_nondonors_ids.add(did)
        elif amt>0 and prop_amt> amt:
            sincere_donors_ids.add(did)
        else:
            # print(amt,prop_amt)
#             sincere_donors_ids.add(did)
            sincere_nondonors_ids.add(did)

print("Sincere donors= ",len(sincere_donors_ids))
print("Sincere non-donors= ",len(sincere_nondonors_ids))

Sincere donors=  115
Sincere non-donors=  185


In [60]:
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

In [61]:
print('No of valid data-points {}'.format(len(df[df.B4==1])))

import scipy.stats

def group_name_dict(id_):
    if id_ in sincere_donors_ids:
        return 'SD'
    elif id_ in sincere_nondonors_ids:
        return 'SND'

def print_label_dist(label_name):
    labels_list  = list(set(list(df[df.B4==1][label_name])))
    labels_dict  = ddict(lambda: ddict(int))
    convs_dict   = ddict(lambda: ddict(list))
        
    df2 = df[df.B4 ==1]
    curr_id = list(df2['B2'])[0]
    temp_labels_dict = ddict(int)
    
    for index, row in df2.iterrows():
        if curr_id != row['B2']:
            group_name = group_name_dict(curr_id)
            sum_labels_list = np.sum([temp_labels_dict[i] for i in temp_labels_dict])
            for label in labels_list:
                if label in temp_labels_dict:
                    convs_dict[group_name][label].append(temp_labels_dict[label]/sum_labels_list)
                else:
                    convs_dict[group_name][label].append(0)
            curr_id = row['B2']
            temp_labels_dict = ddict(int)
        temp_labels_dict[row[label_name]]+=1
        labels_dict[group_name_dict(row['B2'])][row[label_name]]+=1
    
    # meant for the last index which would have been missed otherwise
    group_name = group_name_dict(curr_id)
    sum_labels_list = np.sum([temp_labels_dict[i] for i in temp_labels_dict])
    for label in labels_list:
        if label in temp_labels_dict:
            convs_dict[group_name][label].append(temp_labels_dict[label]/sum_labels_list)
        else:
            convs_dict[group_name][label].append(0)

    print('Distribution of the different strategies among the different categories')
    print("Label\t\t\t#Donor #NonDonor Mean-D Mean-ND pval-1\t pval-2\t Cohen-d")
    for label in labels_list:
        a = convs_dict['SD'][label]
        b = convs_dict['SND'][label]
        mu_a = round(np.mean(a),3)
        mu_b = round(np.mean(b),3)
        sum_a = labels_dict['SD'][label]
        sum_b = labels_dict['SND'][label]
        stat, pval   = scipy.stats.ks_2samp(a,b)
        stat2,pval2  = scipy.stats.ttest_ind(a,b,equal_var=False)
        effect_size  = round(cohen_d(a,b),4)
        if len(label)<=len('personal-choice'):
            print("{}\t\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(label, sum_a, sum_b, mu_a , mu_b, round(pval,4), round(pval2,4), effect_size))
        else:
            print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(label, sum_a, sum_b, mu_a , mu_b, round(pval,4), round(pval2,4), effect_size))
    print('*********************************\n\n')
    return 

No of valid data-points 4638


In [62]:
print_label_dist('fine_labels')

print_label_dist('coarse_labels')

print_label_dist('resistance_labels')

Distribution of the different strategies among the different categories
Label			#Donor #NonDonor Mean-D Mean-ND pval-1	 pval-2	 Cohen-d
not-a-strategy		1482	2017	0.799	0.717	0.001	0.0012	0.3851
direct-rejection	6	45	0.004	0.015	0.2815	0.007	-0.2925
delay-tactic		16	40	0.011	0.014	0.3824	0.5072	-0.0844
personal-choice		42	104	0.023	0.036	0.9201	0.0662	-0.2026
attack-credibility	44	100	0.025	0.033	0.6147	0.3106	-0.1186
hesitance		9	26	0.004	0.01	0.994	0.0557	-0.2001
deflect-responsibility	38	73	0.021	0.028	0.9796	0.2926	-0.1199
nitpicking		13	30	0.008	0.009	1.0	0.9187	-0.0121
organization-inquiry	137	322	0.088	0.117	0.0071	0.0763	-0.2145
self-pity		31	63	0.017	0.022	0.8255	0.4917	-0.0871
*********************************


Distribution of the different strategies among the different categories
Label			#Donor #NonDonor Mean-D Mean-ND pval-1	 pval-2	 Cohen-d
empowerment		73	167	0.04	0.057	0.3824	0.0942	-0.1993
avoidance		31	111	0.019	0.04	0.0028	0.0036	-0.3373
contesting		232	525	0.142	0.1